In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Batch Prediction with Model Monitoring

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_monitoring/batch_prediction_model_monitoring.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_monitoring/batch_prediction_model_monitoring.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

In this notebook, you will learn how to use Model Monitoring with batch prediction requests on a deployed Vertex AI Model resource. In a companion notebook, <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/model_monitoring.ipynb" target="_blank">Vertex AI Model Monitoring with Explainable AI Feature Attributions</a>, you can learn about how to apply model monitoring to streaming, real-time predictions.

### Objective
This tutorial uses the following Google Cloud ML services:

- Vertex AI Model Monitoring
- Vertex AI Batch Prediction
- Vertex AI Model resource

The steps performed include:

- Upload a pre-trained model as a Vertex AI Model resource.
- Generate batch prediction requests.
- Interpret the statistics, visualizations, other data reported by the model monitoring feature.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertext AI
* Google Cloud Storage

Learn about [Vertext AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### What is Vertex AI batch prediction?

<a href="https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions" target="_blank">Batch Prediction</a> is a service that processes a collection (i.e. a batch) of machine learning inference requests in bulk, with less stringent response time requirements than real-time or streaming predictions.

### What is Vertex AI model monitoring?

<a href="https://cloud.google.com/vertex-ai/docs/model-monitoring" target="_blank">Model Monitoring</a> is a service that automatically determines 
whether production machine learning traffic differs from  training data, or varies substantially over time, in terms of model predictions or feature attributions. When that happens, you can be alerted automatically and responsively, whereby, you can detect model decay which may negatively impact your operations -- such as your customer experience and/or revenue.

### The example model

The model you'll use in this notebook is based on [this blog post](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml). The idea behind this model is that your company has extensive log data describing how your game users have interacted with the site. The raw data contains the following categories of information:

- identity - unique player identitity numbers
- demographic features - information about the player, such as the geographic region in which a player is located
- behavioral features - counts of the number of times a  player has triggered certain game events, such as reaching a new level
- churn propensity - this is the label or target feature, it provides an estimated probability that this player will churn, i.e. stop being an active player.

The blog article referenced above explains how to use BigQuery to store the raw data, pre-process it for use in machine learning, and train a model. Because this notebook focuses on model monitoring, rather than training models, you're going to reuse a pre-trained version of this model, which has been exported to Google Cloud Storage. In the next section, you will setup your environment and import this model into your own project.

## Installation

Install the packages required for executing this notebook.

In [ ]:
import os
import sys

assert sys.version_info.major == 3, "This notebook requires Python 3."

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install Python package dependencies.
! pip3 install -q tensorflow-data-validation $USER_FLAG
! pip3 install -q google-api-core $USER_FLAG
! pip3 install -q google-cloud-aiplatform $USER_FLAG

### Restart the kernel

After you install the SDK, you need to restart the notebook kernel so it can find the packages. You can restart kernel from *Kernel -> Restart Kernel*, or running the following:

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Set up your Google Cloud project

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.7

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.


In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "python-docs-samples-tests"  # @param {type:"string"}

In [ ]:
! gcloud config set project $PROJECT_ID

#### Set your email address
This is used for delivering model monitoring notifications.


In [ ]:
EMAIL_ADDRESS = "[your-email-address]"  # @param {type:"string"}
if not EMAIL_ADDRESS or EMAIL_ADDRESS == "[your-email-address]":
    print("EMAIL_ADDRESS not specified, please correct before proceeding.")

#### Set your region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.


In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# Determine notebook environment.
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)
IS_COLAB = "google.colab" in sys.modules

# If on Vertex AI Workbench, then don't execute this code
if not IS_WORKBENCH_NOTEBOOK and not IS_USER_MANAGED_WORKBENCH_NOTEBOOK:
    if IS_COLAB:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Upload the model

The churn propensity model you'll be using in this notebook has been trained in BigQuery ML and exported to a Google Cloud Storage bucket. This illustrates how you can easily export a trained model and move a model from one cloud service to another. 

Next, import the model. **If you've already imported your model, you can skip this step.**

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [ ]:
from datetime import datetime
import json
import time
import re
import tensorflow as tf
import tensorflow_data_validation as tfdv
from tensorflow_data_validation.utils import io_util 
from tensorflow_metadata.proto.v0 import statistics_pb2

MODEL_DISPLAY_NAME=f"batch_prediction_monitoring_test_model_{datetime.now().strftime('%Y%m%d%H%M%S')}"
CONTAINER_IMAGE_URI="us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-4:latest"
ARTIFACT_URI="gs://mco-mm/churn"

output = ! gcloud ai models upload \
  --region=$REGION \
  --display-name=$MODEL_DISPLAY_NAME \
  --artifact-uri=$ARTIFACT_URI \
  --container-image-uri=$CONTAINER_IMAGE_URI \
  --format="value(model)"
MODEL_ID = output[1].split("/")[5]
print(f"Model {MODEL_ID} created.")

## Submit a batch prediction request with model monitoring enabled

### Create Cloud Storage buckets

This cell creates two Cloud Storage buckets:

- `gs://PROJECT_ID_bp_mm_input` contains the batch prediction request data.
- `gs://PROJECT_ID_bp_mm_output` contains the batch prediction output as well as the model monitoring results.

In [ ]:
# Copy files to your projects gs bucket to avoid permission issues.
# Ignore any error(s) for bucket already exists.
OUTPUT_GS_PATH = f"gs://{PROJECT_ID.replace('-', '_')}_bp_mm_output"
INPUT_GS_PATH = f"gs://{PROJECT_ID.replace('-', '_')}_bp_mm_input"
PUBLIC_TRAINING_DATASET = "gs://bp_mm_public_data/churn/churn_bp_insample.csv"
TRAINING_DATASET = f"{INPUT_GS_PATH}/churn_bp_insample.csv"
TRAINING_DATASET_FORMAT = "csv"

! gsutil mb -p {PROJECT_ID} -l {REGION} -b on {INPUT_GS_PATH}
! gsutil mb -p {PROJECT_ID} -l {REGION} -b on {OUTPUT_GS_PATH}
! gsutil copy $PUBLIC_TRAINING_DATASET $INPUT_GS_PATH

### Create batch prediction job

This step parameterizes and builds the data structure representing the batch prediction request, with model monitoring enabled.

The BatchPredictionJob object specifies the input source, the data format, and the computing resources requests for the batch prediction. Learn more about <a href="https://cloud.google.com/vertex-ai/docs/samples/aiplatform-create-batch-prediction-job-sample" target="_blank">BatchPredictionJob</a>.

The ModelMonitoringConfig object specifies the alerting email address, the training dataset, the features to be monitored, and their associated alerting thresholds. Learn more about <a href="https://cloud.google.com/vertex-ai/docs/model-monitoring" target="_blank">ModelMonitoringConfig</a>.

In [ ]:
now = datetime.now()
INPUT_URI = "gs://bp_mm_public_data/churn/churn_bp_outsample.jsonl"
OUTPUT_URI = OUTPUT_GS_PATH
INSTANCES_FORMAT = "jsonl"
PREDICTIONS_FORMAT = "jsonl"
JOB_NAME_PREFIX = "bp_mm_demo"
MODEL_NAME = f"projects/{PROJECT_ID}/locations/{REGION}/models/{MODEL_ID}"
MACHINE_TYPE = "n1-standard-8"
BATCH_PREDICTION_JOB_NAME = JOB_NAME_PREFIX + "_" + now.strftime("%Y%m%d%H%M%S")

from google.cloud.aiplatform_v1beta1.types import (
    BatchDedicatedResources, BatchPredictionJob, GcsDestination, GcsSource,
    MachineSpec, ModelMonitoringAlertConfig, ModelMonitoringConfig,
    ModelMonitoringObjectiveConfig, ThresholdConfig)

batch_prediction_job = BatchPredictionJob(
    display_name=BATCH_PREDICTION_JOB_NAME,
    model=MODEL_NAME,
    input_config=BatchPredictionJob.InputConfig(
        instances_format=INSTANCES_FORMAT, gcs_source=GcsSource(uris=[INPUT_URI])
    ),
    output_config=BatchPredictionJob.OutputConfig(
        predictions_format=PREDICTIONS_FORMAT,
        gcs_destination=GcsDestination(output_uri_prefix=OUTPUT_URI),
    ),
    dedicated_resources=BatchDedicatedResources(
        machine_spec=MachineSpec(machine_type=MACHINE_TYPE),
        starting_replica_count=1,
        max_replica_count=1,
    ),
    # Model monitoring service will be triggerred if provide following configs.
    model_monitoring_config=ModelMonitoringConfig(
        alert_config=ModelMonitoringAlertConfig(
            email_alert_config=ModelMonitoringAlertConfig.EmailAlertConfig(
                user_emails=[EMAIL_ADDRESS]
            )
        ),
        objective_configs=[
            ModelMonitoringObjectiveConfig(
                training_dataset=ModelMonitoringObjectiveConfig.TrainingDataset(
                    data_format=TRAINING_DATASET_FORMAT,
                    gcs_source=GcsSource(uris=[TRAINING_DATASET]),
                ),
                training_prediction_skew_detection_config=ModelMonitoringObjectiveConfig.TrainingPredictionSkewDetectionConfig(
                    skew_thresholds={
                        "cnt_user_engagement": ThresholdConfig(value=0.001),
                        "julianday": ThresholdConfig(value=0.001),
                    }
                ),
            )
        ],
    ),
)

### Submit batch prediction job

This step submits the batch prediction request created in the previous step. If successful, it returns a JSON document summarizing the request, which is displayed in the cell output below.

In [ ]:
from google.cloud.aiplatform_v1beta1.services.job_service import \
    JobServiceClient

API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"
client = JobServiceClient(client_options={"api_endpoint": API_ENDPOINT})
out = client.create_batch_prediction_job(
    parent=f"projects/{PROJECT_ID}/locations/{REGION}",
    batch_prediction_job=batch_prediction_job,
)
BATCH_PREDICTION_JOB_ID = out.name.split("/")[-1]
print("BATCH_PREDICTION_JOB_ID:", BATCH_PREDICTION_JOB_ID)

## Verify prediction results

The batch prediction request will be completed after about **17 mins**, and the model monitoring result will be available **10 mins** after that. The request below obtains the batch prediction job id, which is a unique number associated with asynchronous requests like this one.

In [ ]:
# If auto-testing, wait for request completion
if os.getenv("IS_TESTING"):
    time.sleep(1800)

### Browse storage bucket

Click on the link below, which opens the Cloud Storage object viewer on the output bucket you created above, to see the results of your batch prediction request.

When everything is ready, you'll see two folders in this bucket (the precise names will vary):

- `prediction-batch_prediction_monitoring_test_model_20220714100939-2022_07_14T03_10_10_069Z` - this folder contains the your batch prediction results, i.e. the predictions produced by your model for each input in the batch
- `job-8794345073597743104` - this folder contains the model monitoring results, including the model schema, monitoring thresholds and other config settings, statistics, and anomalies

*Note:* that until the request and the monitoring job are completed, you may see empty or partial results in this bucket.

In [ ]:
print(f"https://console.cloud.google.com/storage/browser/{OUTPUT_URI.lstrip('gs://')}")

### Visualize the batch prediction result

Run the following cell to examine the batch prediction results with a tabular and visual analysis using the <a href="https://www.tensorflow.org/tfx/data_validation/get_started" target="_blank">TensorFlow Data Validation</a> package

In [ ]:
! rm -f ./training_stats.pb
! rm -f ./prediction_stats.pb

TRAINING_STATS_SUBPATH = "stats_training/stats/training_stats"
PREDICTION_STATS_SUBPATH = "stats_and_anomalies/stats/current_stats"
STATS_GCS_FOLDER = OUTPUT_URI = (
    OUTPUT_GS_PATH + "/job-" + BATCH_PREDICTION_JOB_ID + "/bp_monitoring/"
)
TRAINING_STATS_GCS_PATH = STATS_GCS_FOLDER + TRAINING_STATS_SUBPATH
print("Looking up statistics from: " + TRAINING_STATS_GCS_PATH)
PREDICTION_STATS_GCS_PATH = STATS_GCS_FOLDER + PREDICTION_STATS_SUBPATH
print("Looking up statistics from: " + PREDICTION_STATS_GCS_PATH)

! gsutil cp $TRAINING_STATS_GCS_PATH ./training_stats.pb
! gsutil cp $PREDICTION_STATS_GCS_PATH ./prediction_stats.pb


# util function to load stats binary file from GCS
def load_stats_binary(input_path):
    stats_proto = statistics_pb2.DatasetFeatureStatisticsList()
    stats_proto.ParseFromString(
        io_util.read_file_to_string(input_path, binary_mode=True)
    )
    return stats_proto


tfdv.visualize_statistics(load_stats_binary("./training_stats.pb"))
tfdv.visualize_statistics(load_stats_binary("./prediction_stats.pb"))

### Check skew results

Finally, you can check the skew results by examining a file in the output bucket. The JSON file contains a report indicating how much the batch prediction data deviates from the training data, on a feature-by-feature basis.

In [ ]:
SKEW_GS_PATH = (
    STATS_GCS_FOLDER
    + "stats_and_anomalies/anomalies/training_prediction_skew_anomalies"
)
! gsutil cat $SKEW_GS_PATH

## Clean up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Delete model resource
! gcloud ai models delete $MODEL_NAME --quiet

# Delete Cloud Storage resources
! gsutil -m rm -r $INPUT_GS_PATH
! gsutil -m rm -r $OUTPUT_GS_PATH

## Learn more...

**Congratulations!** You've now learned how to monitor batch predictions, and how to find and interpret the results. Check out the following resources to learn more about model monitoring and ML Ops.

- [TensorFlow Data Validation](https://www.tensorflow.org/tfx/guide/tfdv)
- [Data Understanding, Validation, and Monitoring At Scale](https://blog.tensorflow.org/2018/09/introducing-tensorflow-data-validation.html)
- [Vertex Product Documentation](https://cloud.google.com/vertex-ai)
- [Vertex AI Model Monitoring Reference Docs](https://cloud.google.com/vertex-ai/docs/reference)
- [Vertex AI Model Monitoring blog article](https://cloud.google.com/blog/topics/developers-practitioners/monitor-models-training-serving-skew-vertex-ai)